In [1]:
import pip

In [2]:
pip.main(["install", "gensim"])

0

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import gensim
import re

C:\Users\vande\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


'pattern' package not found; tag filters are not available for English


In [10]:
!wget https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
train = pd.read_csv("labeledTrainData.tsv", sep="\t")
test = pd.read_csv("testData.tsv", sep="\t")

labels = train.sentiment

train.drop(["id", "sentiment"], axis=1, inplace=True)
test.drop(["id"], axis=1, inplace=True)

In [20]:
def clean_text(text):
    regex = "[^a-zA-Z0-9 ]"
    cleaned_text = re.sub(regex, "", text).lower()
    
    """You may add stemming here"""
    
    return cleaned_text

assert clean_text("ASDASDAsdads.,,   asd,.ads012") == 'asdasdasdads   asdads012'

In [73]:
all = pd.concat([train, test], axis=0)

all["review"] = all["review"].apply(clean_text)

In [22]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300-SLIM.bin.gz", binary=True)

loading projection weights from GoogleNews-vectors-negative300-SLIM.bin.gz
loaded (299567, 300) matrix from GoogleNews-vectors-negative300-SLIM.bin.gz


In [74]:
%%time

def get_vector(words):
    words = list(filter(lambda word: word in model.vocab.keys(), words.split()))
    
    if not words:
        return np.zeros(shape=vectors[0].shape[0])
    
    return np.mean([model[word] for word in words], axis=0)

review_vectors = all.review.apply(get_vector)

Wall time: 38.2 s


In [75]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

tfidf = TfidfVectorizer()
_ = tfidf.fit_transform(all.review)

Wall time: 15.3 s


In [76]:
idfs = list(tfidf.idf_)
vocab = tfidf.vocabulary_

idfs = {key:idfs[value] for key, value in vocab.items()}

def get_vector(words):
    words = list(filter(lambda word: word in model.vocab.keys() and word in tfidf.vocabulary_.keys(), words.split()))
    
    if not words:
        return np.zeros(shape=vectors[0].shape[0])
    
    tf = [vocab[word] / len(words) for word in words]
    weights = np.array([idfs[word] for word in words])
    word_vecs = [model[word] * tf[i] for i, word in enumerate(words)]
    
    return np.multiply(word_vecs, weights.reshape(-1, 1)).sum(axis=0)

In [77]:
%%time
review_vectors = all.review.apply(get_vector)

Wall time: 1min 38s


In [78]:
del idfs
del vocab
del tfidf
del all

In [79]:
w2vs = np.vstack(review_vectors.values)
del review_vectors

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(C=1e-4)
print(cross_val_score(lr, w2vs[:train.shape[0]], labels, scoring="roc_auc").mean())

0.9102913850487356


In [81]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1e-4)
lr.fit(w2vs[:train.shape[0]], labels)
preds = lr.predict_proba(w2vs[train.shape[0]:])

In [82]:
def submit(preds):
    submission = pd.read_csv("sampleSubmission.csv")
    submission.sentiment = preds
    submission.to_csv("submission.csv", index=False)

In [83]:
submit(preds)